In [2]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F


In [3]:
spark = SparkSession.builder.appName('trades-crunch').getOrCreate()

21/11/14 22:49:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [5]:
trades = spark.read.option('header', 'true').csv(
    './airflow/data/raw/trades/year=2020/month=04/day=10/*.csv'
).withColumn('symbol', F.split(F.input_file_name(), '__').getItem(2))\
    .withColumn('date', F.to_date(
        F.split(
            F.split(F.input_file_name(), '__').getItem(3),
            '\.').getItem(0)
    ))

In [6]:
trades.selectExpr('avg(qty)').show()

+------------------------+
|avg(CAST(qty AS DOUBLE))|
+------------------------+
|      1693.3061446781935|
+------------------------+



In [7]:
trades.show(5)

+---+----------+------------+----------+-------------+------------+-----------+------+----------+
| id|     price|         qty|  quoteQty|         time|isBuyerMaker|isBestMatch|symbol|      date|
+---+----------+------------+----------+-------------+------------+-----------+------+----------+
|  0|0.00002989|  4.00000000|0.00011956|1586491200000|       False|       True|SOLBTC|2020-04-10|
|  1|0.00011899| 30.00000000|0.00356970|1586491200424|       False|       True|SOLBTC|2020-04-10|
|  2|0.00011899|130.00000000|0.01546870|1586491200591|       False|       True|SOLBTC|2020-04-10|
|  3|0.00011899| 20.00000000|0.00237980|1586491200895|       False|       True|SOLBTC|2020-04-10|
|  4|0.00011423| 62.00000000|0.00708226|1586491201343|        True|       True|SOLBTC|2020-04-10|
+---+----------+------------+----------+-------------+------------+-----------+------+----------+
only showing top 5 rows



In [12]:
from pyspark.sql.window import Window
minutes = lambda m: m*60*1000

w = (Window.orderBy(F.col("time").cast('long')).rangeBetween(-minutes(5), 0))
trades_2 = trades.withColumn('ma_qty', F.avg('qty').over(w))

trades_2.select('qty', 'ma_qty').show(25)

21/11/14 22:50:55 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+--------------+------------------+
|           qty|            ma_qty|
+--------------+------------------+
| 1568.00000000|            1568.0|
| 4906.00000000|            3237.0|
|16003.00000000| 7492.333333333333|
|16002.00000000|           9619.75|
|16003.00000000|           10896.4|
|15989.00000000|11745.166666666666|
|15994.00000000|12352.142857142857|
|16097.00000000|          12820.25|
|15996.00000000|13173.111111111111|
|16002.00000000|           13456.0|
|16103.00000000|13696.636363636364|
|15990.00000000|          13887.75|
|16000.00000000|15090.416666666666|
|15993.00000000|15159.846153846154|
|15993.00000000|15219.357142857143|
|15995.00000000|15271.066666666668|
|16001.00000000|        15316.6875|
|15996.00000000| 15356.64705882353|
|15993.00000000|           15392.0|
|15990.00000000|15423.473684210527|
|15690.00000000|           15436.8|
| 8865.00000000|          15634.75|
|  498.00000000|             498.0|
|  379.00000000|             438.5|
|  455.00000000|            